In [1]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np
import pickle

C:\Users\djaym7\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def loadData(id):
    with open(f'C:\\Users\\djaym7\\Desktop\\Github\\EmotionRecognition\\data 02\\y_train_{id}','rb') as f:
        y_train=np.array(pickle.load(f))
    with open(f'C:\\Users\\djaym7\\Desktop\\Github\\EmotionRecognition\\data 02\\y_test_{id}','rb') as f:
        y_test=np.array(pickle.load(f))
    with open(f'C:\\Users\\djaym7\\Desktop\\Github\\EmotionRecognition\\data 02\\X_train_{id}','rb') as f:
        X_train = np.array(pickle.load(f))
    with open(f'C:\\Users\\djaym7\\Desktop\\Github\\EmotionRecognition\\data 02\\X_test_{id}','rb') as f:
        X_test = np.array(pickle.load(f))

    

    from sklearn.preprocessing import LabelBinarizer
    encoder = LabelBinarizer()
    y_train = encoder.fit_transform(y_train)
    y_test = encoder.fit_transform(y_test)


    print('X_train:',X_train.shape)
    print('X_test:',X_test.shape)
    print('y_train',y_train.shape) 
    print('y_test',y_test.shape) 

   
    return X_train,y_train,X_test,y_test


In [3]:
#m = 10000
#dataset, human_vocab, machine_vocab, inv_machine_vocab = load_dataset(m)
X_train,y_train,X_test,y_test = loadData(1001)


X_train: (7360, 500, 13)
X_test: (82, 500, 13)
y_train (7360, 6)
y_test (82, 6)


In [4]:
Tx = 500
Ty = 1
#X, Y, Xoh, Yoh = preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty)

In [5]:
# Defined shared layers as global variables
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor = Dense(1, activation = "relu")
activator = Activation('softmax', name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)

In [6]:
def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attetion) LSTM cell
    """
    
    ### START CODE HERE ###
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a" (≈ 1 line)
    s_prev = repeator(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis (≈ 1 line)
    concat = concatenator([a, s_prev])
    # Use densor to propagate concat through a small fully-connected neural network to compute the "energies" variable e. (≈1 lines)
    e = densor(concat)
    # Use activator and e to compute the attention weights "alphas" (≈ 1 line)
    alphas = activator(e)
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell (≈ 1 line)
    context = dotor([alphas, a])
    ### END CODE HERE ###
    
    return context

In [7]:
n_a = 64
n_s = 128
post_activation_LSTM_cell = LSTM(n_s, return_state = True) #initialized globally (post-attention layer)
output_layer = Dense(6, activation='softmax') # final dense-softmax layer

In [8]:
def model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    """
    Arguments:
    Tx -- length of the input sequence #13
    Ty -- length of the output sequence #6
    n_a -- hidden state size of the Bi-LSTM #64
    n_s -- hidden state size of the post-attention LSTM #128
    human_vocab_size -- size of the python dictionary "human_vocab"
    machine_vocab_size -- size of the python dictionary "machine_vocab"

    Returns:
    model -- Keras model instance
    """
    
    # Define the inputs of your model with a shape (Tx,)
    # Define s0 and c0, initial hidden state for the decoder LSTM of shape (n_s,)
    X = Input(shape=(Tx, 13))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
    
    ### START CODE HERE ###
    
    # Step 1: Define your pre-attention Bi-LSTM. Remember to use return_sequences=True. (≈ 1 line)
    a = Bidirectional(LSTM(n_a, return_sequences=True))(X)  #64
    
    # Step 2: Iterate for Ty steps
    for t in range(Ty):
    
        # Step 2.A: Perform one step of the attention mechanism to get back the context vector at step t (≈ 1 line)
        context = one_step_attention(a, s) # a=output of preLSTM, s= zero vector(initially) of size postlstm units n_s=128
        
        # Step 2.B: Apply the post-attention LSTM cell to the "context" vector.
        # Don't forget to pass: initial_state = [hidden state, cell state] (≈ 1 line)
        s, _, c = post_activation_LSTM_cell(context, initial_state=[s, c])
        
        # Step 2.C: Apply Dense layer to the hidden state output of the post-attention LSTM (≈ 1 line)
        out = output_layer(s)
        
        # Step 2.D: Append "out" to the "outputs" list (≈ 1 line)
        outputs.append(out)
    
    # Step 3: Create model instance taking three inputs and returning the list of outputs. (≈ 1 line)
    model = Model([X, s0, c0], outputs)
    
    ### END CODE HERE ###
    
    return model

In [9]:
model = model(Tx, Ty, n_a, n_s, len(human_vocab), len(machine_vocab))

In [10]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 500, 13)      0                                            
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 128)          0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 500, 128)     39936       input_1[0][0]                    
__________________________________________________________________________________________________
repeat_vector_1 (RepeatVector)  (None, 500, 128)     0           s0[0][0]                         
__________________________________________________________________________________________________
concatenat

In [11]:
### START CODE HERE ### (≈2 lines)
opt = Adam(lr = 0.005, beta_1=0.9, beta_2=0.999, decay = 0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
### END CODE HERE ###

In [12]:
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(Yoh.swapaxes(0,1))

In [13]:
model.fit([X_train, s0, c0], y_train,validation_data=(X_test,y_test), epochs=1, batch_size=100)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 3 array(s), but instead got the following list of 1 arrays: [array([[[-3.60436534e+01,  0.00000000e+00, -3.20763172e-14, ...,
         -3.27769755e-13,  0.00000000e+00,  3.59484803e-13],
        [-3.60436534e+01,  0.00000000e+00, -3.20763172e-14, ...,
        ...